In [1]:
from typing import Tuple, Dict, Any, Optional, List
from collections import OrderedDict
import pyrogram.types
from pyrogram.types import Message, User, Chat, MessageReactions, Reaction
from pyrogram.enums import ChatType, MessageMediaType
import os
import pyrogram
from pymongo import MongoClient
from enum import Enum
import datetime
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
NullFormatter = ticker.NullFormatter

MONGO_HOST="mongodb://127.0.0.1:27017/"
MONGO_INITDB_ROOT_USERNAME="root"
MONGO_INITDB_ROOT_PASSWORD="example"
MONGO_DATABASE="messages_new"

conn = MongoClient(MONGO_HOST, username=MONGO_INITDB_ROOT_USERNAME, password=MONGO_INITDB_ROOT_PASSWORD)
db = conn[MONGO_DATABASE]
messages = db["messages"]

In [2]:
result = messages.aggregate([
    {
        '$group': {
            '_id': {
                'chat_id': '$chat.id', 
                'chat_title': '$chat.title'
            }
        }
    }
])
result = [x["_id"] for x in result]
channels = {x["chat_id"]: x["chat_title"] for x in result}
del result
print(channels)

{-1001547361186: 'Carmel News - חדשות מברית המועצות לשעבר וישראל', -1001254976833: 'קול החדשות ב 🆃🅴🅻🅴🅶🆁🅰️🅼🔴', -1001825077227: 'Israel now', -1001435013468: 'مطالعات رژیم صهیونیستی 🇵🇸', -1001428757842: 'בן כספית Ben Caspit - הערוץ הרשמי', -1001282622805: '⚡️🇮🇱חדשות הבזק🇮🇱⚡️', -1001731396712: "חדשות תיבת נח🌈Noah's Ark news", -1001559024378: 'ישראל ימין עכשיו • ISRARL NOW GAZA גל פתוח', -1001728075361: 'صحيفة الحدث الفلسطيني', -1001855522803: '🔸کیسان ابوعمره🔸', -1001375169718: 'מסביב לעולם - Around the World', -1001848822478: '🔥מרכבות האש🔥 - דיווחים בזמן אמת', -1001006165084: '🎬 بیسیمچی مدیا', -1001056678368: 'Tankograd 🛡', -1001960770844: 'אלעאלם אלערבי-العالم العربي| ערן מלכא', -1001588534673: "הפינצטה-השקעות לאנג'לים וכשירים", -1001117292289: 'افـــسر جنـــگ نـــرم', -1001282192381: 'תשואת יתר - וול סטריט', -1001425850587: 'אבו צאלח הדסק הערבי', -1001401013486: '🔸ידיעות בני ברק🔸', -1001346732056: 'اسکواد ایران | Squad Iran', -1001373818759: 'اخبار نظامی ایران و جهان', -1001170925964: '

In [3]:
# find channels with arabic letters
arabic_letters = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"
arabic_channels = {}
for channel_id, channel_name in channels.items():
    if any(letter in arabic_letters for letter in channel_name):
        arabic_channels[channel_id] = channel_name

print(arabic_channels)

{-1001435013468: 'مطالعات رژیم صهیونیستی 🇵🇸', -1001728075361: 'صحيفة الحدث الفلسطيني', -1001855522803: '🔸کیسان ابوعمره🔸', -1001006165084: '🎬 بیسیمچی مدیا', -1001960770844: 'אלעאלם אלערבי-العالم العربي| ערן מלכא', -1001117292289: 'افـــسر جنـــگ نـــرم', -1001346732056: 'اسکواد ایران | Squad Iran', -1001373818759: 'اخبار نظامی ایران و جهان', -1001170925964: 'محمد مهدی بابایی', -1001394985064: 'الناشط حمزة المصري', -1002136406625: 'WAR ROOM | اتاق جنگ', -1001050775424: '🇮🇷وقایع الاتفاقیه نظامی🇮🇷', -1001592257625: 'پارتیزان پلاس', -1001008407570: 'Partisan / پارتیزان', -1001018424368: 'جنگاوران', -1001207026910: 'INTELLITIMES | إنتل تايمز | אינטלי טיימס', -1001342537743: 'اخبار امنیتی - نظامى', -1001613270320: '{شهابا رصدا الإخبارية} ٢٤/٧', -1001970614754: 'میلیتاری در پارسی', -1001132293108: 'خبر سپاه پاسداران', -1001002400952: 'شبكة قدس الإخبارية', -1001683457734: 'Sirat | صِراط', -1001953086336: 'پرایتوری 🕊', -1001628243607: '\u200c\u200cساندیس میم | SANDIS MEME', -1001917130438: 'Al-A

In [4]:
hebrew_letters = "אבגדהוזחטיכלמנסעפצקרשתץףךןם"
hebrew_channels = {}
for channel_id, channel_name in channels.items():
    if any(letter in hebrew_letters for letter in channel_name):
        hebrew_channels[channel_id] = channel_name

print(hebrew_channels)

{-1001547361186: 'Carmel News - חדשות מברית המועצות לשעבר וישראל', -1001254976833: 'קול החדשות ב 🆃🅴🅻🅴🅶🆁🅰️🅼🔴', -1001428757842: 'בן כספית Ben Caspit - הערוץ הרשמי', -1001282622805: '⚡️🇮🇱חדשות הבזק🇮🇱⚡️', -1001731396712: "חדשות תיבת נח🌈Noah's Ark news", -1001559024378: 'ישראל ימין עכשיו • ISRARL NOW GAZA גל פתוח', -1001375169718: 'מסביב לעולם - Around the World', -1001848822478: '🔥מרכבות האש🔥 - דיווחים בזמן אמת', -1001960770844: 'אלעאלם אלערבי-العالم العربي| ערן מלכא', -1001588534673: "הפינצטה-השקעות לאנג'לים וכשירים", -1001282192381: 'תשואת יתר - וול סטריט', -1001425850587: 'אבו צאלח הדסק הערבי', -1001401013486: '🔸ידיעות בני ברק🔸', -1001586914912: 'צבא החדשות', -1001168948411: 'העין השביעית', -1001402297849: 'איילת לאש', -1001654802453: 'בודקים', -1001147703577: '🚨 חדשות ישראל ללא צנזורה 🚨', -1001561122816: 'יהודים מדווחים - הערוץ הרשמי', -1001143765178: 'אבו עלי אקספרס', -1001207026910: 'INTELLITIMES | إنتل تايمز | אינטלי טיימס', -1001952721314: 'בז news - חדשות צבא וביטחון', -1001521365